In [8]:
import requests, os, csv, io, zipfile, shutil
import pandas as pd

In [12]:
# Read the CSV file with cleaned metadata (email content) related to influencer, celeb, celebrity, social media, launch, agency, VIP
df = pd.read_csv('data\celeb.txt', delimiter='\t') #pd.read_csv('data\influencer.csv')
ids_list = df['id'].tolist()

In [13]:
print(len(ids_list))

1642


Run to extract OCR text (last column) from zip folder with all CSV files - Extracting CSV is not working on VCL due to limted storage stops after extracting CSV 286

In [ ]:
zip_file_path = 'data\JUUL_Labs_Collection.zip'

def unzip_and_rezip(zip_file_path, ids_list):
    ids_set = set(ids_list)
    temp_directory = "temp_extracted_files"
    new_csv_name = "influencer_ocr.csv"
    output_csv_path = os.path.join('\data', new_csv_name)  # Saves new CSV in '/data' directory

    # Extract all files to a temporary directory
    with zipfile.ZipFile(zip_file_path, 'r') as z:
        z.extractall(temp_directory)

    # Create the new CSV file to store ocr text for the relevant ids
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['id', 'ocr text'])

        # Loop through each CSV file
        for file_name in os.listdir(temp_directory):
            full_path = os.path.join(temp_directory, file_name)
            if full_path.endswith('.csv'):
                with open(full_path, 'r', newline='', encoding='utf-8') as file:
                    reader = csv.reader(file, delimiter='|')
                    for row in reader:
                        if row[0] in ids_set:
                            writer.writerow([row[0], row[-1]])  # Write only ID and the last column (ocr text)

    # Rezip the original CSV files
    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as z:
        for file_name in os.listdir(temp_directory):
            full_path = os.path.join(temp_directory, file_name)
            if file_name != new_csv_name:  # Exclude the new influencer_ocr.csv
                z.write(full_path, arcname=file_name)

    # Clean up the temporary directory
    shutil.rmtree(temp_directory)

unzip_and_rezip(zip_file_path, ids_list)

Attempt to download required OCR text - works but takes a long time for 86938 documents. 
No space left on device error.

In [14]:
base_url = 'https://download.industrydocuments.ucsf.edu/'
output_csv_path = 'data\ocr_celeb_texts.csv'

# Write IDs and OCR text to new CSV
with open(output_csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'ocr_text'])

    # Loop through the document IDs and download OCR files
    for ids in ids_list:
        url_path = f"{ids[0]}/{ids[1]}/{ids[2]}/{ids[3]}/{ids}"
        download_url = f"{base_url}{url_path}/{ids}.ocr"

        # Download the OCR file
        response = requests.get(download_url)
        if response.status_code == 200:
            # Save the OCR content to a temporary file
            temp_filename = f"{ids}.ocr"
            with open(temp_filename, 'wb') as f:
                f.write(response.content)
            
            # Read the OCR text
            try:
                with open(temp_filename, 'r') as file:
                    ocr_text = file.read()
                # Write to the CSV file
                writer.writerow([ids, ocr_text])
            except Exception as e:
                pass

            # Delete the temporary ocr file
            os.remove(temp_filename)
        else:
            pass

In [ ]:
#Sanity Check 
id_count = 0
with open('data\ocr_celeb_texts.csv', mode='r', newline='') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        id_count += 1
print(id_count)

In [ ]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import bigrams, trigrams
from collections import Counter

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

# Dictionary to store OCR texts
ocr_texts = {}

# Path to the CSV file
csv_path = 'data\ocr_texts.csv'

# Read the OCR text from CSV into the dictionary
with open(csv_path, mode='r', newline='') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header
    for row in reader:
        doc_id, ocr_text = row
        ocr_texts[doc_id] = ocr_text

# Preprocess each stored OCR text
for doc_id, ocr_text in ocr_texts.items():
    processed_text = preprocess_text(ocr_text)
    ocr_texts[doc_id] = processed_text

In [ ]:
print(len(ocr_texts))

Count Vectorizer

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

documents = list(ocr_texts.values())  # Get all the pre-processed text values as a list

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

doc_term_matrix = vectorizer.fit_transform(documents)

# Convert the matrix to a DataFrame
df = pd.DataFrame(doc_term_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Save the DataFrame to a CSV file
csv_output_path = 'data\doc_term_matrix.csv'
df.to_csv(csv_output_path, index=False)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

documents = list(ocr_texts.values())

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Convert the TF-IDF matrix to a DataFrame
df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Save the DataFrame to a CSV file
csv_output_path = '/content/tfidf_matrix.csv'
df.to_csv(csv_output_path, index=False)

Working codes below - not final

Customize stopwords 
1. Most frequent words
2. Words appearing in n% of documents

In [15]:
stop_words = set(stopwords.words('english'))
#custom_stopwords = []
stop_words.update(custom_stopwords)

In [ ]:
stop_words = set(stopwords.words('english'))
print(stop_words)

In [ ]:
# Count the frequency of each word
word_counts = Counter(processed_text.split())
most_common_words = word_counts.most_common(100)  # Get the top 10 most common words

print(most_common_words)

In [ ]:
!pip install wordcloud matplotlib
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# Create the word cloud
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='black',
                min_font_size = 10).generate(processed_text)

# View the word cloud:
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

In [23]:
words = word_tokenize(processed_text)

In [26]:
# Bi-grams
bi_grams = list(bigrams(words))
# Tri-grams
tri_grams = list(trigrams(words))

bi_gram_counts = Counter(bi_grams)
tri_gram_counts = Counter(tri_grams)

In [ ]:
# Print the counts
print("Bi-gram counts:")
for bi_gram, count in bi_gram_counts.items():
    print(f"{bi_gram}: {count}")

In [ ]:
print("\nTri-gram counts:")
for tri_gram, count in tri_gram_counts.items():
    print(f"{tri_gram}: {count}")

In [ ]:
# Top N bi-grams
top_N_bi_grams = bi_gram_counts.most_common(50)

# Print the top 20 bi-grams
print("Top 20 Bi-grams:")
for bi_gram, count in top_N_bi_grams:
    print(f"{bi_gram}: {count}")


In [ ]:
# Top N tri-grams
top_N_tri_grams = tri_gram_counts.most_common(50)

# Print the top 20 tri-grams
print("\nTop N Tri-grams:")
for tri_gram, count in top_N_tri_grams:
    print(f"{tri_gram}: {count}")